In [1]:
import scipy
scipy.__version__

'1.1.0'

# Objective 4. Import the point cloud into Python and do some basic computation.

Note. In Python we will use a range of libraries. A short description will
be provided, but for detailed instructions use the library documentation and
Google.

In [2]:
# -*- coding: utf-8 -*-
"""
Computes point neighbourshood parameters and removes irrelevant points.

@author: Chris Lucas
"""

import pandas as pd
import time
import os
from scipy.spatial import KDTree
from data_preprocessing import las_to_csv, downsample
from par_computation import neighbourhood_features

# %% file paths
las_path = "L:/ws_MachineLearning/ChrisLucas/delineating-linear-elements/Data/ResearchArea.las"
las2txt_path = "L:/ARun/LAStools/bin/las2txt.exe"
CloudCompare_path = "L:/ARun/CloudCompare_v2.9.1_bin_x64/CloudCompare.exe"

In [3]:
# %% Prepare data and load into python
# downsample point cloud and convert to csv
las = downsample(las_path, 0.3, tool_path=CloudCompare_path)
csv_path = las_to_csv(las, method='las2txt', tool_path=las2txt_path)

Subsampled output file already exists.
CSV output file already exists.


To read in the very large ASCII ﬁle we will use the pandas library (comes
already installed with Anaconda). Pandas is a library that provides high
performance and easy to use data structures.
## Assignment 11. Import pandas and use it to read in the point cloud csv ﬁle.

In [4]:
# Load the csv point cloud file
print "Loading point cloud csv file using pandas.."
csv_path="L:/ws_MachineLearning/ChrisLucas/delineating-linear-elements/Data/ResearchArea_sub_0_3.csv"
point_cloud = pd.read_csv(csv_path, delimiter=';', header=None,
                          names=['X', 'Y', 'Z', 'intensity',
                                 'return_number', 'number_of_returns'])

points = point_cloud.as_matrix(columns=['X', 'Y', 'Z'])



Loading point cloud csv file using pandas..


d:\miniconda3\envs\py2.7\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


# Objective 5. Use a k-d tree data structure to compute nearest neighbours.
When processing point clouds it’s often needed to compute the nearest
neighbours of a point. For example when computing neighbourhood param-
eters or when using a region growing algorithm. To eﬃciently compute these
nearest points a data structure can be used. Diﬀerent data structures exist,
including the k-d tree, octree and R-tree. The most eﬃcient data structure
depends on the data and the application. We will us a k-d tree, but this is
certainly not the only option.
The python library SciPy (comes already installed with Anaconda) con-
tains many functions for scientiﬁc computing. It includes a spatial module,
which contains an algorithm for the construction of k-d trees.

## Assignment 14. Import cKDTree from SciPy and use it to construct a k-d tree for the point cloud.
Note. SciPy also has a KDTree function. The diﬀerence between cKDTree
and KDTree is that KDTree is coded in pure python while cKDTree is coded
in Cython (a version of python which gives python-like code C-like perfor-
mance). Consequently cKDTree is signiﬁcantly faster than KDTree.

In [5]:
# %% Compute nearest neighbours
print "Computing nearest neighbours.."
neighbours = [50]
kdtree = KDTree(points)
print('oooooooooooooo')

Computing nearest neighbours..
oooooooooooooo


Now that the k-d tree is constructed it can be queried for neighbours.
This can be done in two ways: (i) loop over the points and query for each
point separately, and (ii) query every point at once. Generally the former is
more memory eﬃcient, while the latter is more CPU eﬃcient.
Before we query for neighbours we need to deﬁne our neighbourhood.
This can be done in three ways: (i) k nearest neighbours, (ii) spherical, and
(iii) cylindrical.

## Assignment 15. Use the k-d tree to compute the neighbourhood of a point using the three diﬀerent neighbourhood deﬁnitions.


In [6]:
distances, point_neighbours = kdtree.query(points, max(neighbours))
print "Done!"

Done!


# Objective 6. Use a structure tensor to compute neighbourhood parameters.

In [7]:
# %% Compute point features
features = ['delta_z', 'std_z', 'radius', 'density', 'norm_z',
            'linearity', 'planarity', 'sphericity', 'omnivariance',
            'anisotropy', 'eigenentropy', 'sum_eigenvalues',
            'curvature']
feature_values = {}

In [8]:
for k in neighbours:
    print "Computing covariance features.."
    t = time.time()
    fv = neighbourhood_features(points, point_neighbours[:, :k],
                                features, distances[:, :k])
    print "Done! Runtime: %s" % str(time.time() - t)
    feature_values[k] = fv

Computing covariance features..
 Computing structure tensors..
 Done!
 Computing features..
 Done!
Done! Runtime: 101.585999966


In [9]:
for k in neighbours:
    for i, f in enumerate(features):
        key = f + '_' + str(k)
        point_cloud[key] = pd.Series(feature_values[k][:, i])

In [10]:
# %% Trim the data by deleting all non scatter points from the point cloud
print "Trimming data.."
point_cloud.query('sphericity_50 > 0.05 & planarity_50 < 0.7', inplace=True)
point_cloud.reset_index(drop=True, inplace=True)
print "Done!"

Trimming data..
Done!


In [11]:
# %% Compute normalized return number
point_cloud['norm_returns'] = (point_cloud['return_number'] /
                               point_cloud['number_of_returns'])

In [12]:
# %% Output data
las_path_root = os.path.splitext(las_path)[0]
out_filename = '%s_params.csv' % (las_path_root)
print(out_filename)
point_cloud.to_csv(out_filename, index=False)

L:/ws_MachineLearning/ChrisLucas/delineating-linear-elements/Data/ResearchArea_params.csv
